In [1]:
%pip install mediapipe opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2 
import mediapipe as mp

In [15]:
cap = cv2.VideoCapture("Download.mp4")
face_mesh = mp.solutions.face_mesh.FaceMesh()

lipsUpperOuter = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291]
lipsLowerOuter = [146, 91, 181, 84, 17, 314, 405, 321, 375, 291]
lipsUpperInner = [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308]
lipsLowerInner = [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308]
lips = list(set(lipsUpperOuter + lipsLowerOuter + lipsUpperInner + lipsLowerInner))

while True:    
    check, test_image = cap.read()
    if check is not True:
        break
    
    height, width, _ = test_image.shape 

    rgb_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

    results = face_mesh.process(rgb_image)
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
                for i in lips:
                    pt = face_landmarks.landmark[i]
                    x = int(pt.x * width)
                    y = int(pt.y * height)
                    cv2.circle(test_image, (x, y), 1, (100, 100, 0), -1)

    cv2.imshow("Result",test_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()